# Functions

In [1]:
import pandas as pd

# Define a function to check if the panel data is balanced
def check_panel_balanced(df, entity_col, time_col):
    """
    Checks if a pandas DataFrame represents a balanced panel.

    Args:
        df (pd.DataFrame): The DataFrame to check.
        entity_col (str): The name of the column that identifies each entity (e.g., 'id', 'firm_id').
        time_col (str): The name of the column that identifies each time period (e.g., 'year', 'date').

    Returns:
        bool: True if the panel is balanced, False otherwise.
    """
    # Group the DataFrame by the entity and count the number of observations for each
    counts = df.groupby(entity_col)[time_col].count()

    # A panel is balanced if all entities have the same number of observations.
    # This is true if the number of unique counts is exactly one.
    return counts.nunique() == 1

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels.panel import PooledOLS, PanelOLS, RandomEffects
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy import stats

# --------------------------
# Hausman robusto a singularidad
# --------------------------
def hausman(fe_res, re_res):
    b = fe_res.params
    B = re_res.params
    common = b.index.intersection(B.index)
    b, B = b[common], B[common]
    Vb = fe_res.cov.loc[common, common]
    VB = re_res.cov.loc[common, common]
    diff = (b - B).to_numpy()
    S = (Vb - VB).to_numpy()
    S_pinv = np.linalg.pinv(S)
    stat = float(diff.T @ S_pinv @ diff)
    dof = int(np.linalg.matrix_rank(S))
    pval = 1.0 - stats.chi2.cdf(stat, dof) if dof > 0 else np.nan
    return {"stat": stat, "df": dof, "pval": float(pval)}

# --------------------------
# F-tests LSDV para efectos fijos
# --------------------------
def _lsdv_and_Ftests(df, y_var, x_vars, test_entity=True, test_time=True):
    y = df[y_var]
    X_base = df[x_vars]

    parts = [X_base]
    ent_cols, tim_cols = [], []

    if test_entity:
        ent = pd.get_dummies(df.index.get_level_values(0), drop_first=True, prefix="ent")
        parts.append(ent)
        ent_cols = list(ent.columns)
    if test_time:
        tim = pd.get_dummies(df.index.get_level_values(1), drop_first=True, prefix="time")
        parts.append(tim)
        tim_cols = list(tim.columns)

    # Reset de índices para concatenar
    parts_reset = [p.reset_index(drop=True) for p in parts]
    X_lsdv = pd.concat(parts_reset, axis=1)
    X_lsdv = sm.add_constant(X_lsdv).astype(float)

    # Reset y
    y_numeric = y.reset_index(drop=True).astype(float)

    # Ajuste OLS
    ols = sm.OLS(y_numeric, X_lsdv).fit()

    # F-test auxiliar
    def ftest_zero(cols):
        if len(cols) == 0:
            return {"F": np.nan, "pval": np.nan, "df_num": 0, "df_den": int(ols.df_resid)}
        k = len(ols.model.exog_names)
        R = np.zeros((len(cols), k))
        name_to_pos = {n: i for i, n in enumerate(ols.model.exog_names)}
        for r, col in enumerate(cols):
            R[r, name_to_pos[col]] = 1.0
        ft = ols.f_test(R)
        F = float(np.squeeze(ft.fvalue))
        p = float(np.squeeze(ft.pvalue))
        return {"F": F, "pval": p, "df_num": len(cols), "df_den": int(ols.df_resid)}

    f_ent = ftest_zero(ent_cols)
    f_tim = ftest_zero(tim_cols)
    f_both = ftest_zero(ent_cols + tim_cols)

    return ols, f_ent, f_tim, f_both

# --------------------------
# Diagnósticos para panel
# --------------------------
def diagnosticos_panel(df, y_var, x_vars):
    y = df[y_var]
    X = df[x_vars]
    Xc = sm.add_constant(X)

    # VIF solo numéricas
    num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    vif_df = pd.DataFrame({
        "Variable": num_cols,
        "VIF": [variance_inflation_factor(X[num_cols].values, i) for i in range(len(num_cols))]
    })

    # Breusch–Pagan (orientativo)
    ols = sm.OLS(y.astype(float), Xc.astype(float)).fit()
    bp = het_breuschpagan(ols.resid, ols.model.exog)
    bp_out = {"LM": float(bp[0]), "pval": float(bp[1])}

    # Durbin–Watson (orientativo)
    dw = float(sm.stats.durbin_watson(ols.resid))

    print("\n=== Diagnósticos datos de entrada ===")
    print("\n= Multicolinealidad (VIF): =")
    print("""
*VIF = 1 → No hay correlación entre la variable y las demás (ideal).
*1 < VIF < 5 → Correlación moderada, no suele ser un problema grave.
*VIF ≥ 5 → Indica correlación alta, empieza a ser preocupante.
*VIF ≥ 10 → Multicolinealidad severa, la variable probablemente está aportando información redundante y puede distorsionar el modelo.
          """)
    print(vif_df)
    
    print("\n = Heterocedasticidad =")
    print("\nHeterocedasticidad (Breusch–Pagan, orientativo):", bp_out)
     # Interpretación del test de Breusch-Pagan
    if bp[1] < 0.05:
        print("Heterocedasticidad detectada")
    else:
        print("No se detecta heterocedasticidad")

    # Interpretación del test de Durbin–Watson
    print("\n = Correlación entre resuidos del modelo orientativo =")
    print(f"\nDurbin–Watson (orientativo): {dw:.4f}")
    if dw < 1.5:
        print("Posible autocorrelación positiva")
    elif dw > 2.5:
        print("Posible autocorrelación negativa")
    else:
        print("No hay evidencia de autocorrelación")

    return {"VIF": vif_df, "BP": bp_out, "DW": dw}

# --------------------------
# Función principal de selección de modelo panel
# --------------------------
def seleccionar_modelo_panel(df, y_var, x_vars, cov_type="clustered", cluster_entity=True, cluster_time=False):
    if not isinstance(df.index, pd.MultiIndex):
        raise ValueError("El DataFrame debe tener MultiIndex (entity, time).")

    y = df[y_var]
    X = df[x_vars]

    # Ajuste modelos
    pooled = PooledOLS(y, sm.add_constant(X)).fit(cov_type=cov_type,
                                                  cluster_entity=cluster_entity,
                                                  cluster_time=cluster_time)

    fe_ent = PanelOLS(y, X, entity_effects=True).fit(cov_type=cov_type,
                                                     cluster_entity=cluster_entity,
                                                     cluster_time=cluster_time)

    fe_tim = PanelOLS(y, X, time_effects=True).fit(cov_type=cov_type,
                                                   cluster_entity=cluster_entity,
                                                   cluster_time=cluster_time)

    fe_bth = PanelOLS(y, X, entity_effects=True, time_effects=True).fit(cov_type=cov_type,
                                                                        cluster_entity=cluster_entity,
                                                                        cluster_time=cluster_time)

    # RE con constante explícita
    re = RandomEffects(y, sm.add_constant(X)).fit(cov_type=cov_type,
                                                  cluster_entity=cluster_entity,
                                                  cluster_time=cluster_time)

    # F-tests LSDV
    _, f_ent, f_tim, f_bth = _lsdv_and_Ftests(df, y_var, x_vars, test_entity=True, test_time=True)

    # Hausman FE_entidad vs RE
    haus = hausman(fe_ent, re)

    # Selección automática
    if f_bth["pval"] < 0.05:
        seleccionado = "FE_ambos"
        elegido = fe_bth
    elif f_ent["pval"] < 0.05 and f_tim["pval"] >= 0.05:
        seleccionado = "FE_entidad" if haus["pval"] < 0.05 else "RE"
        elegido = fe_ent if haus["pval"] < 0.05 else re
    elif f_tim["pval"] < 0.05 and f_ent["pval"] >= 0.05:
        seleccionado = "FE_tiempo"
        elegido = fe_tim
    else:
        seleccionado = "PooledOLS"
        elegido = pooled

    print("\n=== Selección de modelo ===")
    print(f"Modelo recomendado: {seleccionado}")
    print(f"Hausman FE_entidad vs RE: Chi2={haus['stat']:.4f}, df={haus['df']}, p={haus['pval']:.4f}")
    
    if haus['pval'] >= 0.05:
        print ("""
Interpretacion: "No se rechaza H₀: No hay evidencia de correlación entre efectos no observados y variables explicativas." 
El modelo de Efectos Aleatorios (RE) es consistente y eficiente.
               """) 
    else:
        print ("""
Interpretacion: "Se rechaza H₀: Hay evidencia de correlación entre efectos no observados y variables explicativas." 
El modelo de Efectos Fijos (FE) es más adecuado.
               """) 

    # Diagnósticos
    diags = diagnosticos_panel(df, y_var, x_vars)

    return {
        "modelos": {
            "PooledOLS": pooled,
            "FE_entidad": fe_ent,
            "FE_tiempo": fe_tim,
            "FE_ambos": fe_bth,
            "RE": re
        },
        "tests": {
            "F_LSDV_entidad": f_ent,
            "F_LSDV_tiempo": f_tim,
            "F_LSDV_ambos": f_bth,
            "Hausman_FE_vs_RE": haus
        },
        "diagnosticos": diags,
        "Seleccionado": seleccionado,
        "Resultado_elegido": elegido
    }


# About Dataset
## Dataset Details:
Data Set contains Cost Data for U.S. Airlines, 90 Observations On 6 Firms For 15 Years, 1970-1984

Predictors:
I = Airline,

T = Year,

Q = Output, in revenue passenger miles, index number,

PF = Fuel price,

LF = Load factor, the average capacity utilization of the fleet.

Response:
C = Total cost, in $1000,

Acknowledgements and Credit
These data are a subset of a larger data set provided to the author by Professor Moshe Kim.
They were originally constructed by Christensen Associates of Madison, Wisconsin.

Inspiration
Perform various econometric analyses to check which model suits best for the given dataset. To start with can check this notebook which is programmed in R.

# 1. Getting to know the dataset

In [3]:
data = pd.read_csv('PanelData.csv')

In [4]:
data.head()

,I,T,C,Q,PF,LF
0,1,1,1140640,0.952757,106650,0.534487
1,1,2,1215690,0.986757,110307,0.532328
2,1,3,1309570,1.091980,110574,0.547736
3,1,4,1511530,1.175780,121974,0.540846
4,1,5,1676730,1.160170,196606,0.591167


In [5]:
data.rename(columns = {'I':'Airline', 
                       'T':'Year', 
                       'C':'Response', 
                       'Q': 'Revenue', 
                       'PF': 'Fuel_price',
                       'LF': 'Load_factor'}, 
                       inplace = True)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Airline      90 non-null     int64  
 1   Year         90 non-null     int64  
 2   Response     90 non-null     int64  
 3   Revenue      90 non-null     float64
 4   Fuel_price   90 non-null     int64  
 5   Load_factor  90 non-null     float64
dtypes: float64(2), int64(4)
memory usage: 4.3 KB


In [7]:
data.describe()

,Airline,Year,Response,Revenue,Fuel_price,Load_factor
count,90.000000,90.000000,9.000000e+01,90.000000,9.000000e+01,90.000000
mean,3.500000,8.000000,1.122524e+06,0.544995,4.716830e+05,0.560460
std,1.717393,4.344698,1.192075e+06,0.533586,3.295029e+05,0.052793
min,1.000000,1.000000,6.897800e+04,0.037682,1.037950e+05,0.432066
25%,2.000000,4.000000,2.920460e+05,0.142128,1.298475e+05,0.528806
50%,3.500000,8.000000,6.370010e+05,0.305028,3.574335e+05,0.566085
75%,5.000000,12.000000,1.345968e+06,0.945278,8.498398e+05,0.594658
max,6.000000,15.000000,4.748320e+06,1.936460,1.015610e+06,0.676287


# 2. Getting to know the data

## 2.1 Is our Panel Data Balanced?

In [8]:
check_panel_balanced(data, 
                     'Airline',
                     'Year')

True

# 3 Based on the data, Carry out a Panel Data Analysis

In [10]:
data_panel = data.set_index(['Airline', 'Year'])

In [11]:
data_panel

Response   Revenue  Fuel_price  Load_factor
Airline Year                                             
1       1      1140640  0.952757      106650     0.534487
        2      1215690  0.986757      110307     0.532328
        3      1309570  1.091980      110574     0.547736
        4      1511530  1.175780      121974     0.540846
        5      1676730  1.160170      196606     0.591167
...                ...       ...         ...          ...
6       11      381478  0.112640      874818     0.517766
        12      506969  0.154154     1013170     0.580049
        13      633388  0.186461      930477     0.556024
        14      804388  0.246847      851676     0.537791
        15     1009500  0.304013      819476     0.525775

[90 rows x 4 columns]

In [12]:
# ===== 2. Usar seleccionar_modelo_panel =====
resultados = seleccionar_modelo_panel(data_panel, 
                                      "Response", 
                                      [x for x in data_panel.columns if x not in ['Response']])
# ===== 3. Acceder a resultados =====
print("\nModelo seleccionado:", resultados["Seleccionado"])
print("\nResumen del modelo seleccionado:")
print(resultados["Resultado_elegido"].summary)


=== Selección de modelo ===
Modelo recomendado: FE_ambos
Hausman FE_entidad vs RE: Chi2=74.6768, df=3, p=0.0000

Interpretacion: "Se rechaza H₀: Hay evidencia de correlación entre efectos no observados y variables explicativas." 
El modelo de Efectos Fijos (FE) es más adecuado.
               

=== Diagnósticos datos de entrada ===

= Multicolinealidad (VIF): =

*VIF = 1 → No hay correlación entre la variable y las demás (ideal).
*1 < VIF < 5 → Correlación moderada, no suele ser un problema grave.
*VIF ≥ 5 → Indica correlación alta, empieza a ser preocupante.
*VIF ≥ 10 → Multicolinealidad severa, la variable probablemente está aportando información redundante y puede distorsionar el modelo.
          
      Variable       VIF
0      Revenue  2.268502
1   Fuel_price  3.547414
2  Load_factor  4.239845

 = Heterocedasticidad =

Heterocedasticidad (Breusch–Pagan, orientativo): {'LM': 24.082163253333643, 'pval': 2.4012506412653555e-05}
Heterocedasticidad detectada

 = Correlación entre res

This panel data analysis concludes that a **Fixed Effects model with both entity and time effects (FE_ambos)** is the most appropriate for the dataset. This conclusion is based on key diagnostic tests that address the specific characteristics of the panel data.

---

### Key Findings and Justification

* **Model Selection**: The analysis rejects simpler models in favor of the more complex Fixed Effects model. The **Hausman test** result of a Chi-squared value of 74.6768 with a p-value of 0.0000 confirms that a Fixed Effects model is superior to a Random Effects model, as there is significant evidence of a correlation between the unobserved effects and the independent variables[cite: 1, 2].
* **Significance of Effects**: The **F-test for Poolability** also shows a highly significant p-value of 0.0000, which indicates that a standard pooled OLS model is not adequate and that the data requires accounting for either entity-specific, time-specific, or both types of effects.
* **Model Fit**: The selected model has a robust R-squared value of **0.8137**, suggesting that the included variables effectively explain a large portion of the variance in the dependent variable.

---

### Variable Performance and Data Diagnostics

* **Significant Variables**: Of the three independent variables, only **Revenue** is a statistically significant predictor of the dependent variable (`Response`), with a highly significant p-value of 0.0000.
* **Non-Significant Variables**: The coefficients for **Fuel_price** and **Load_factor** are not statistically significant (p-values of 0.1519 and 0.2379, respectively), indicating that they do not have a significant impact on the dependent variable within this model.
**Multicollinearity**: The Variance Inflation Factor (VIF) values for all variables are below 5[cite: 3, 4, 5, 6]. This indicates that while there is some moderate correlation among the predictors, it is not severe enough to distort the model's coefficients.
**Heteroscedasticity and Autocorrelation**: The **Breusch–Pagan test** detected heteroscedasticity, while the **Durbin–Watson statistic** (0.4342) points to a potential positive autocorrelation in the residuals[cite: 1]. The use of a `Clustered` covariance estimator in the final model helps to correct for these issues, providing more reliable standard errors.